ecg_fractal_analysis is intendeed to create pickle datasets in the form: normal ecg signals x 12 ecg derivations, containig the values of the corresponding complexity measures

Load requiered packages

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import neurokit2 as nk

Read normal_ecg_age dataframe containing normal ecg id's and corresponding age

In [2]:
normal_ecg_age = pd.read_pickle('normal_ecg_age.pickle')

Age clases based on intervals define on the age feature with labels from 0: [10, 20), 1: [20, 30), 2: [30, 40), 3: [40, 50), 4: [50, 60), 5: [60, 70), 6: [70, 80), 7: [80, 90), 8: [90, 100)

In [3]:
normal_ecg_age.head()

,ECG_ID,Age,Age_class
0,A00002,32,2
1,A00003,63,5
2,A00006,46,3
3,A00008,32,2
4,A00009,48,3


Selecting signals length for fracta analysis in order to standarized calculations due to fractal features are related with signal's length

Including in the dataframe signals length requiered for complexity calculations. If OSError then signal's length is set to NaN. Needed for standarizing signals length because complexity measures depends on signal's length

In [4]:
length_dataframe = pd.DataFrame(columns=['ID', 'I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
temp_length = []
for id in normal_ecg_age['ECG_ID']:
    with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
        for der in range(12):
            try:
                temp_length.append(len(f['ecg'][der]))
            except OSError:
                temp_length.append(np.NaN)
                continue
    temp_length.insert(0, id)
    length_dataframe.loc[len(length_dataframe)] = temp_length
    temp_length = []          

In [5]:
length_dataframe.head()

,ID,I,II,III,aVR,aVL,aVF,V1,V2,V3,V4,V5,V6
0,A00002,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000
1,A00003,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500
2,A00006,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
3,A00008,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
4,A00009,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000


Create normal_ecg_length dataframe containing the derivations lenghts

In [6]:
length_dataframe.to_pickle('normal_ecg_length.pickle')

Verifying missing data in dataframe. NaN values related with OSError in signals reading. Solution: download SPH dataset again.

In [7]:
length_dataframe.isnull().sum()

ID     0
I      0
II     0
III    0
aVR    0
aVL    0
aVF    0
V1     0
V2     0
V3     0
V4     0
V5     0
V6     0
dtype: int64

Minimun and maximun signal lengths. Signal length set to 5000 for all derivations

In [8]:
pd.concat([length_dataframe.min(), length_dataframe.max()], axis=1)

,0,1
ID,A00002,A25765
I,5000,17500
II,5000,17500
III,5000,17500
aVR,5000,17500
aVL,5000,17500
aVF,5000,17500
V1,5000,17500
V2,5000,17500
V3,5000,17500


Complexity Katz's fractal dimension measure 

In [9]:
Katz_dataframe = pd.DataFrame(columns=['ID', 'I_Katz', 'II_Katz', 'III_Katz', 'aVR_Katz', 'aVL_Katz', 'aVF_Katz', 'V1_Katz', 'V2_Katz', 'V3_Katz', 'V4_Katz', 'V5_Katz', 'V6_Katz'])
kfd_temp = []
for id in normal_ecg_age['ECG_ID']:
        with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
            for der in range(12):
                try:
                    kfd, _ = nk.fractal_katz(f['ecg'][der][:5000])
                    kfd_temp.append(kfd) 
                except OSError:
                    kfd_temp.append(np.NaN)
                    continue
        kfd_temp.insert(0, id)            
        Katz_dataframe.loc[len(Katz_dataframe)] = kfd_temp
        kfd_temp = []

In [11]:
Katz_dataframe.to_pickle('normal_ecg_katz.pickle')

Complexity LL's fractal dimension measure 

In [12]:
LL_dataframe = pd.DataFrame(columns=['ID', 'I_LL', 'II_LL', 'III_LL', 'aVR_LL', 'aVL_LL', 'aVF_LL', 'V1_LL', 'V2_LL', 'V3_LL', 'V4_LL', 'V5_LL', 'V6_LL'])
LL_temp = []
for id in normal_ecg_age['ECG_ID']:
        with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
            for der in range(12):
                try:
                    LL, _ = nk.fractal_linelength(f['ecg'][der][:5000])
                    LL_temp.append(LL) 
                except OSError:
                    LL_temp.append(np.NaN)
                    continue
        LL_temp.insert(0, id)            
        LL_dataframe.loc[len(LL_dataframe)] = LL_temp
        LL_temp = []

In [13]:
LL_dataframe.to_pickle('normal_ecg_ll.pickle')

Complexity Petrosian's fractal dimension measure

In [14]:
Pt_dataframe = pd.DataFrame(columns=['ID', 'I_Pt', 'II_Pt', 'III_Pt', 'aVR_Pt', 'aVL_Pt', 'aVF_Pt', 'V1_Pt', 'V2_Pt', 'V3_Pt', 'V4_Pt', 'V5_Pt', 'V6_Pt'])
Pt_temp = []
for id in normal_ecg_age['ECG_ID']:
        with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
            for der in range(12):
                try:
                    Pt, _ = nk.fractal_petrosian(f['ecg'][der][:5000], symbolize='C', show=False)
                    Pt_temp.append(Pt) 
                except OSError:
                    Pt_temp.append(np.NaN)
                    continue
        Pt_temp.insert(0, id)            
        Pt_dataframe.loc[len(Pt_dataframe)] = Pt_temp
        Pt_temp = []

In [15]:
Pt_dataframe.to_pickle('normal_ecg_petrosian.pickle')

Complexity Sevcik's fractal dimension measure

In [16]:
Sk_dataframe = pd.DataFrame(columns=['ID', 'I_Sk', 'II_Sk', 'III_Sk', 'aVR_Sk', 'aVL_Sk', 'aVF_Sk', 'V1_Sk', 'V2_Sk', 'V3_Sk', 'V4_Sk', 'V5_Sk', 'V6_Sk'])
Sk_temp = []
for id in normal_ecg_age['ECG_ID']:
        with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
            for der in range(12):
                try:
                    Sk, _ = nk.fractal_sevcik(f['ecg'][der][:5000])
                    Sk_temp.append(Sk)
                except OSError:
                    Sk_temp.append(np.NaN)
                    continue
        Sk_temp.insert(0, id)            
        Sk_dataframe.loc[len(Sk_dataframe)] = Sk_temp
        Sk_temp = []

In [17]:
Sk_dataframe.to_pickle('normal_ecg_sevcik.pickle')

Complexity NLD's fractal dimension measure

In [18]:
NLD_dataframe = pd.DataFrame(columns=['ID', 'I_NLD', 'II_NLD', 'III_NLD', 'aVR_NLD', 'aVL_NLD', 'aVF_NLD', 'V1_NLD', 'V2_NLD', 'V3_NLD', 'V4_NLD', 'V5_NLD', 'V6_NLD'])
NLD_temp = []
for id in normal_ecg_age['ECG_ID']:
        with h5py.File(f'E:/1-DENIS/Biomarkers/SPH dataset/records/{id}.h5', 'r') as f:
            for der in range(12):
                try:
                    NLD, _ = nk.fractal_nld(f['ecg'][der][:5000], corrected=False)
                    NLD_temp.append(NLD)
                except OSError:
                    NLD_temp.append(np.NaN)
                    continue
        NLD_temp.insert(0, id)            
        NLD_dataframe.loc[len(NLD_dataframe)] = NLD_temp
        NLD_temp = []

nk.fractal_nld(sig, corrected=False)

c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\lib\nanfunctions.py:1739: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr, where=where)
c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\lib\nanfunctions.py:1739: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr, where=where)
c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\YANIBIS\ecg_biomarkers\lib\site-packages\numpy\core\fromnumeric.py:

NameError: name 'sig' is not defined

In [15]:
NLD_dataframe.to_pickle('normal_ecg_nld.pickle')

Complexity XX's fractal dimension measure